<a href="https://colab.research.google.com/github/aditichak22/ml-decision-tree/blob/main/Decision_Tree_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd

def prepareData(datafile):
  data = pd.read_csv(datafile, index_col=False, names=["Y", "x1", "x2", "x3", "x4", "x5", "x6", "x7", "x8", "x9", "x10", "x11", "x12", "x13", "x14", "x15", "x16", "x17", "x18", "x19", "x20", "x21", "x22"])

  data['Y'][data['Y'] == 0] = -1
  X = data.iloc[:, 1:23]
  y = data.iloc[:, 0:1]

  # m = no. of training samples, n = no. of features
  m,n = X.shape
  return data, X, y

data, X, y = prepareData("/content/drive/MyDrive/heart_train.data")
test_data, test_X, test_y = prepareData("/content/drive/MyDrive/heart_test.data")

In [ ]:
data

In [ ]:
data.columns[1]

'x1'

In [ ]:
class Node(object):    
    
    def __init__(self, data, parent_attr_value):
        self.data = data
        self.children = []
        self.attr = None
        self.decision = None
        self.parent_attr = None
        self.parent_attr_value = parent_attr_value

class DT(object):
    
    def __init__(self, data, attr, is_0p, is_all_pos):
        self.data = data
        self.attr = attr
        self.is_0p = is_0p
        self.is_all_pos = is_all_pos
        self.root = None
        self.training_error = None
        
    def classify(self):
        node = Node(self.data, None)
        self.root = node 
        self.build_tree(node)
        
    def build_tree(self, node):        
        data = node.data
        attr = self.attr
        is_0p = self.is_0p
        is_all_pos = self.is_all_pos
        
        if is_all_pos is not None:
            node.decision = 1 if is_all_pos else -1
            node.attr = 'POSITIVE' if is_all_pos else 'NEGATIVE'
            return
        
        attr_values = data[attr].unique()    
        node.attr = attr
        
        for attr_value in attr_values:
            split = data[data[attr] == attr_value]
            child = Node(split, attr_value)
            child.parent_attr = attr
            node.children.append(child)
            
            if attr_value == 0 and is_0p: 
                child.decision = 1
            elif attr_value == 1 and is_0p:
                child.decision = -1
            elif attr_value == 0 and not is_0p:
                child.decision = -1
            elif attr_value == 1 and not is_0p:
                child.decision = 1
                
    
    def predict_dataset(self, data):    
        Y_pred = []
        for i in range(0, len(data)):
            prediction = self.predict(self.root, data.iloc[i])
            Y_pred.append(prediction)
        return Y_pred
    
    def predict(self, node, data_point):        
        if node.decision is not None:
            return node.decision    
        attr = node.attr            
        child = None
        for child in node.children:        
            if child.parent_attr_value == data_point[attr]:
                break              
        return self.predict(child, data_point)      

In [ ]:
def build_hypotheses(data):
    classifiers = []
    
    all_positive = DT(data, None, None, True)
    all_negative = DT(data, None, None, False)
    
    classifiers.append(all_negative)
    classifiers.append(all_positive)
    
    for attr in data.columns:
        if attr == 'Y' or attr == 'W':
            continue
        
        classifier = DT(data, attr, True, None)
        classifiers.append(classifier)
        
        classifier = DT(data, attr, False, None)
        classifiers.append(classifier)
        
    for i in range(len(classifiers)):
        classifiers[i].classify()
        
    return classifiers

def fetchMinAlpha(alphas, classifiers, data, position):
    current_alpha = alphas[position]    
    M,N = data.shape
    
    Y = data['Y'].values.reshape(M, 1)
    Y_pred = np.array(classifiers[position].predict_dataset(data)).reshape(M, 1)
    misclassifications = abs(Y-Y_pred)
    
    misclassifications_indices = np.where(misclassifications > 0)[0]
    correct_classification_indices = np.where(misclassifications <= 0)[0]
    
    misclassified_data = data.iloc[misclassifications_indices]
    correctly_classified_data = data.iloc[correct_classification_indices]
    
    misclassified_data_length = len(misclassifications_indices)
    correctly_classified_data_length = len(correct_classification_indices)
    
    Y_misclassified = data['Y'].iloc[misclassifications_indices].values.reshape((misclassified_data_length,1))
    Y_correctly_classified = data['Y'].iloc[correct_classification_indices].values.reshape((correctly_classified_data_length,1))  
    
    Y_pred_misclassified = np.zeros((misclassified_data_length,1))
    Y_pred_correctly_classified = np.zeros((correctly_classified_data_length, 1))
    
    for i in range(len(alphas)):        
        if i == position:
            continue
        
        alpha = alphas[i]
        Y_pred_misclassified = Y_pred_misclassified + alpha*(np.array(classifiers[i].predict_dataset(misclassified_data)).reshape(misclassified_data_length,1))
        Y_pred_correctly_classified = Y_pred_correctly_classified + alpha*(np.array(classifiers[i].predict_dataset(correctly_classified_data)).reshape(correctly_classified_data_length,1))
        
    correctly_classified = np.exp(-1.0 * Y_correctly_classified * Y_pred_correctly_classified).sum()
    misclassified = np.exp(-1.0 * Y_misclassified * Y_pred_misclassified).sum()
    
    modified_alpha = 0.5 * np.log(correctly_classified/misclassified)
    
    return (alphas, modified_alpha, modified_alpha-current_alpha)
    

def fetchNextMinAdjustedAlphas(alphas, classifiers, data, counter):
    N_c = len(alphas)    
    current = counter + 1
    isRoundRobinComplete = False    
    while not isRoundRobinComplete :
        (alphas, modified_alpha, difference) = fetchMinAlpha(alphas, classifiers, data, (current%N_c))
        print('C ALPHA ', alphas[current%N_c], 'M ALPHA', modified_alpha, 'DIFFERENCE ', difference, ', POSITION ', current)
        if abs(difference) > 1e-4:
            alphas[current%N_c] = modified_alpha
            return (alphas, current%N_c)
        
        if current-counter > N_c:
            isRoundRobinComplete = True
            return (alphas, None)
        
        current = current+1   

def coordinate_descent(data, classifiers):    
    N_c = len(classifiers)
    alphas = np.zeros(N_c).reshape(N_c,1)
    
    iterations = counter = 0
    is_local_optimum = False
    while not is_local_optimum:
        (alphas, counter) = fetchNextMinAdjustedAlphas(alphas, classifiers, data, counter)
        
        iterations = iterations+1
        if counter is None:
            is_local_optimum = True
            break
        
        print('--------------------------------------------------------')
        print('ITERATION ', iterations, ', ATTRIBUTE ', counter)
        print('--------------------------------------------------------')
    
    print('ALPHAS ', alphas)
    
    return alphas

def accuracy(data, alphas, classifiers):
    M, N = data.shape
    Y = data['Y'].values.reshape(M,1)
    Y_pred = np.zeros(M).reshape(M,1)
    
    N_c = len(classifiers)
    
    for i in range(N_c):
        alpha = alphas[i]
        Y_pred = Y_pred + alpha*(np.array(classifiers[i].predict_dataset(data)).reshape(M,1))
        
    
    Y_pred[Y_pred < 0] = -1
    Y_pred[Y_pred >= 0] = 1
    #print(Y-Y_pred)
    
    misclassification = sum(abs(Y-Y_pred))/2
    
    return (1.0 - misclassification/M ) * 100

classifiers = build_hypotheses(data)
alphas = coordinate_descent(data, classifiers)
training_accuracy = accuracy(data, alphas, classifiers)
print('Training accuracy : ', training_accuracy)
test_accuracy = accuracy(test_data, alphas, classifiers)
print('Test accuracy : ', test_accuracy)

        
        

Problem 3


2a) 

Optimal values of alpha [-7.572, -0.013, 2.21, 0, 0.487, 0, 5.279, 0, -0.811, 0,
-0.316, 0, -0.131, 0, -0.6, 0, -4.88, 0, 3.589, 0, -2.079, 0, -0.392, 0, 0.447, 0, -0.794, 0, -3.06, 0, -1.19, 0, -0.744, 0, -2.952, 0, -1.898, 0, -0.05, 0, -0.312, 0 -0.159, 0, -0.377]

Exponential loss - np.exp(-1 * y * pred).sum() = 39.5090

2b)

Training accuracy - 83.54
Testing accuracy - 68.82


2c)



In [ ]:
class Node(object):    
    
    def __init__(self, data, parent_attr_value):
        self.data = data
        self.children = []
        self.attr = None
        self.decision = None
        self.parent_attr = None
        self.parent_attr_value = parent_attr_value

class DT(object):
    
    def __init__(self, data):
        self.data = data
        self.root = None
        self.training_error = None
        
    def classify(self):
        node = Node(self.data, None)
        self.root = node 
        return self.build_tree(node)
        
    def best_attribute(self, node):
        data = node.data
        M, N = data.shape;
        
        minimum_error = 1
        best_attr = None
        for attr in data.columns:
            if attr == 'Y' or attr == 'W':
                continue
            
            A_0pos_error = A_1pos_error = 0
            attr_values = data[attr].unique()    
            for attr_value in attr_values:
                split = data[data[attr] == attr_value]
                
                W_p = (split[split['Y'] == 1]['W']).sum()
                W_n = (split[split['Y'] == -1]['W']).sum()
                
                if attr_value == 0:                
                    A_0pos_error = A_0pos_error + W_n
                    A_1pos_error = A_1pos_error + W_p
                elif attr_value == 1:
                    A_0pos_error = A_0pos_error + W_p
                    A_1pos_error = A_1pos_error + W_n
            
            error = min(A_0pos_error, A_1pos_error)
            if error < minimum_error:
                minimum_error = error
                best_attr = attr
                is_0p = A_0pos_error < A_1pos_error
                
        All_pos_error = data[data['Y'] == -1]['W'].sum()
        All_neg_error = data[data['Y'] == 1]['W'].sum()
        
        All_pos = All_neg = False
        if All_pos_error < minimum_error:
            minimum_error = All_pos_error
            All_pos = True
            All_neg = False
            
        if All_neg_error < minimum_error:
            minimum_error = All_neg_error
            All_pos = False
            All_neg = True
                
        return (best_attr, minimum_error, is_0p, All_pos, All_neg)
        
    def build_tree(self, node):        
        data = node.data
        (attr, error, is_0p, All_pos, All_neg) = self.best_attribute(node)
        
        if All_pos:
            node.decision = 1
            node.attr = 'POSITIVE'
            return error
        
        if All_neg:
            node.decision = -1
            node.attr = 'NEGATIVE'
            return error
        
        attr_values = data[attr].unique()    
        node.attr = attr
        
        for attr_value in attr_values:
            split = data[data[attr] == attr_value]
            child = Node(split, attr_value)
            child.parent_attr = attr
            node.children.append(child)
            
            if attr_value == 0 and is_0p: 
                child.decision = 1
            elif attr_value == 1 and is_0p:
                child.decision = -1
            elif attr_value == 0 and not is_0p:
                child.decision = -1
            elif attr_value == 1 and not is_0p:
                child.decision = 1
                
        return error
    
    def predict_dataset(self, data):    
        Y_pred = []
        for i in range(0, len(data)):
            prediction = self.predict(self.root, data.iloc[i])
            Y_pred.append(prediction)
        return Y_pred
    
    def predict(self, node, data_point):        
        if node.decision is not None:
            return node.decision    
        attr = node.attr            
        child = None
        for child in node.children:        
            if child.parent_attr_value == data_point[attr]:
                break              
        return self.predict(child, data_point)       
         


def update_weights(data, classifier, alpha, error):
    normalizing_constant = (2.0)*(np.sqrt(error*(1-error)))
    
    Y = data['Y']
    Y_pred = classifier.predict_dataset(data)
    W = data['W']
    
    W_updated = ( (W)*(np.exp(-1.0 * Y*Y_pred*alpha))/normalizing_constant )
    data['W'] = W_updated
    
    return data


def adaboost(data_train, iterations):
    # Initialize all weights to (1/M)
    M, N = data_train.shape;
    data = data_train
    data['W'] = (np.ones(M)*(1.0/M)).reshape(M,1)
    
    classifiers = []
    alphas = []
    
    for i in range(iterations):
        # Select the classifier that has minimizes weighted misclassification error
        classifier = DT(data)
        error = classifier.classify()
        
        # Compute alpha
        alpha = (0.5)*(np.log((1-error)/error))
        
        # Update weights
        data = update_weights(data, classifier, alpha, error)
        
        alphas.append(alpha)
        classifiers.append(classifier)
        print('ATTR : ', i, classifier.root.attr)
        
    return (alphas, classifiers)

def predict(data, alphas, classifiers):
    
    M, N = data.shape
    Y = data['Y'].values.reshape(M,1)
    Y_pred = np.zeros(M).reshape(M,1)
    
    N_c = len(classifiers)
    
    for i in range(N_c):
        alpha = alphas[i]
        Y_pred = Y_pred + alpha*(np.array(classifiers[i].predict_dataset(data)).reshape(M,1))
        
    
    Y_pred[Y_pred < 0] = -1
    Y_pred[Y_pred >= 0] = 1
    #print(Y-Y_pred)
    
    misclassification = sum(abs(Y-Y_pred))/2
    
    return (1.0 - misclassification/M ) * 100
    
(alphas, classifiers) = adaboost(data, iterations=20)
training_accuracy = predict(data, alphas, classifiers)
print('ACCURACY ON TRAINING SET', training_accuracy)
test_accuracy = predict(test_data, alphas, classifiers)
print('ACCURACY ON TEST SET : ', test_accuracy)
